# Цветовые модели, морфология, сегментация

In [ ]:
# !pip install scikit-image
# !pip install scikit-learn
# !pip install imutils

In [ ]:
from skimage.color import rgb2hsv, label2rgb
from skimage import data, color, io, img_as_float, feature
from skimage.filters import threshold_otsu, try_all_threshold
from skimage.util import img_as_ubyte, invert

from skimage.transform import hough_circle, hough_circle_peaks, hough_ellipse
from skimage.feature import canny, peak_local_max
from skimage.draw import circle_perimeter, ellipse_perimeter
from skimage.segmentation import clear_border, watershed

from skimage.morphology import erosion, dilation, opening, closing, white_tophat, black_tophat, square
from skimage.morphology import skeletonize, convex_hull_image, medial_axis, thin
from skimage.morphology import disk
from skimage.measure import label, regionprops

import imutils
import cv2

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import deque
from math import sqrt, exp
import numpy as np
import scipy.ndimage as ndi

In [ ]:
import skimage
from skimage import data, restoration, util, img_as_float, filters, color, morphology, segmentation
from skimage.filters import threshold_otsu, try_all_threshold, threshold_niblack, threshold_sauvola, threshold_multiotsu

from skimage.filters import sobel, rank
from skimage.measure import label
from skimage.color import label2rgb

from skimage.segmentation import watershed, mark_boundaries
from skimage.feature import peak_local_max
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
 
from skimage.feature import canny
from skimage.draw import line
from skimage.util import img_as_ubyte

from skimage.data import binary_blobs

from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate

from skimage.filters import sobel
from skimage.measure import label
from skimage.util import img_as_float
from skimage.feature import canny

from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial

from matplotlib import cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import deque
from math import sqrt, exp
import numpy as npshow_binary
import scipy.ndimage as ndi
import pandas as pd
import math

## 1. Цветовые пространства
## 1.1. Переход из RGB в HSV
Обычно объекты на изображениях имеют разные цвета (оттенки) и яркость, поэтому эти функции можно использовать для разделения разных областей изображения. В представлении RGB оттенок и яркость выражаются как линейная комбинация каналов R, G, B, тогда как в модели HSV они соответствуют отдельным каналам изображения (каналы Hue и Value).

In [ ]:
rgb_img = data.coffee()
hsv_img = rgb2hsv(rgb_img)
hue_img = hsv_img[:, :, 0]
value_img = hsv_img[:, :, 2]

fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(8, 2))

ax0.imshow(rgb_img)
ax0.set_title("RGB image")
ax0.axis('off')
ax1.imshow(hue_img, cmap='hsv')
ax1.set_title("Hue channel")
ax1.axis('off')
ax2.imshow(value_img,)
ax2.set_title("Value channel")
ax2.axis('off')

fig.tight_layout()

На основании цветового пространства HSV можно проводить базовую сегментацию. Установим порог на канале Hue, чтобы отделить чашку от фона.

In [ ]:
hue_threshold = 0.04
binary_img = hue_img > hue_threshold

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8, 3))

ax0.hist(hue_img.ravel(), 512)
ax0.set_title("Histogram of the Hue channel with threshold")
ax0.axvline(x=hue_threshold, color='r', linestyle='dashed', linewidth=2)
ax0.set_xbound(0, 0.12)
ax1.imshow(binary_img)
ax1.set_title("Hue-thresholded image")
ax1.axis('off')

fig.tight_layout()

Выполним дополнительную пороговую обработку для канала Value, чтобы частично удалить тень чашки.

In [ ]:
fig, ax0 = plt.subplots(figsize=(4, 3))

value_threshold = 0.10
binary_img = (hue_img > hue_threshold) | (value_img < value_threshold)

ax0.imshow(binary_img)
ax0.set_title("Hue and value thresholded image")
ax0.axis('off')

fig.tight_layout()
plt.show()

В OpenCV доступно более 150 методов преобразования цветового пространства. Но мы рассмотрим только два, которые наиболее широко используются: BGR ↔ Gray и BGR ↔ HSV.

Для преобразования цвета мы используем функцию cv.cvtColor (input_image, flag), где flag определяет тип преобразования.

Для преобразования BGR → Gray мы используем флаг cv.COLOR_BGR2GRAY. Аналогично для BGR → HSV мы используем флаг cv.COLOR_BGR2HSV. 

Рассмотрим существующие флаги:

In [ ]:
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags)

Следует отметить, для HSV диапазон оттенков составляет [0,179], диапазон насыщенности - [0,255], а диапазон значений - [0,255]. В разных программах используются разные масштабы. Поэтому, если вы сравниваете значения OpenCV с ними, необходимо нормализовать эти диапазоны.

## 1.2. Отслеживание объекта на основе модели HSV
Теперь, когда мы знаем, как преобразовать изображение BGR в HSV, мы можем использовать его для извлечения объекта из сцены. В HSV легче представить цвет, чем в цветовом пространстве BGR. Далее мы попытаемся извлечь объект синего цвета:
- регистрируем каждый кадр видео
- преобразовываем из BGR в HSV
- устанавливаем порог изображения HSV для диапазона синего цвета
- извлекаем только синий объект

In [ ]:
cap = cv2.VideoCapture(0)

# define range of blue color in HSV
lower_color = np.array([100, 50, 50])
upper_color = np.array([130, 255, 255])

while(True):
    
    # Take each frame
    _, frame = cap.read()
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('res', res)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()
cap.release()

## 2. Морфологическая обработка
Морфологические преобразования - это группа простых операций, основанных на форме объектов на изображении. Обычно операция выполняется для двоичных изображений, иногда для серошкальных. Для реализации нужно два входа, один - это наше исходное изображение, второй - структурный элемент или ядро, которое определяет характер операции.

In [ ]:
# Rectangular Kernel
kern_rect = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
# Elliptical Kernel
kern_ellip = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
# Cross-shaped Kernel
kern_cross = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1, sharex=ax[0], sharey=ax[0])
ax[1] = plt.subplot(1, 3, 2, sharex=ax[0], sharey=ax[0])
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(kern_rect, cmap=plt.cm.gray)
ax[0].set_title('Rectangular Kernel')

ax[1].imshow(kern_ellip, cmap=plt.cm.gray)
ax[1].set_title('Elliptical Kernel')

ax[2].imshow(kern_cross, cmap=plt.cm.gray)
ax[2].set_title('Cross-shaped Kernel')

plt.show()

Считаем изображение и определим функцию для демонстрации эффектов морфологической обработки

In [ ]:
orig_phantom = img_as_ubyte(data.shepp_logan_phantom())
fig, ax = plt.subplots()
ax.imshow(orig_phantom, cmap=plt.cm.gray)
plt.show()

In [ ]:
def plot_comparison(original, filtered, filter_name):

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')

## 2.1. Эрозия
Морфологическая эрозия устанавливает пиксель в (i, j) на минимум по всем пикселям в окрестности с центром в (i, j). Элемент структурирования, selem, переданный в erosion, представляет собой логический массив, который описывает эту окрестность. Ниже мы используем диск для создания кругового структурирующего элемента, который используется в большинстве следующих примеров.

In [ ]:
selem = disk(6)
eroded = erosion(orig_phantom, selem)
plot_comparison(orig_phantom, eroded, 'erosion')

## 2.2. Дилатация
Морфологическая дилатация устанавливает пиксель в (i, j) на максимум по всем пикселям в окрестности с центром в (i, j). Дилатация увеличивает яркие области и уменьшает темные области

In [ ]:
dilated = dilation(orig_phantom, selem)
plot_comparison(orig_phantom, dilated, 'dilation')

## 2.3. Размыкание
Морфологическое размыкание на изображении определяется как эрозия с последующей дилатацией. Размыкание может удалить небольшие яркие пятна и соединить небольшие темные разрывы.

In [ ]:
opened = opening(orig_phantom, selem)
plot_comparison(orig_phantom, opened, 'opening')

Пример применения размыкания с помощью OpenCV

In [ ]:
im_j = cv2.imread('j_opening.png', 0)
_, im_j = cv2.threshold(im_j, 127, 255, cv2.THRESH_BINARY)
kernel = np.ones((7,7), np.uint8)

opening_j = cv2.morphologyEx(im_j, cv2.MORPH_OPEN, kernel)
plot_comparison(im_j, opening_j, 'opening opencv')

## 2.4. Замыкание
Морфологическое замыкание изображения определяется как дилатация, за которой следует эрозия. Замыкание позволяет удалить небольшие темные пятна и соединить небольшие светлые разрывы.

In [ ]:
phantom = orig_phantom.copy()
phantom[10:30, 200:210] = 0

closed = closing(phantom, selem)
plot_comparison(phantom, closed, 'closing')

Пример применения OpenCV

In [ ]:
im_j = cv2.imread('j_closing.png',0)
_, im_j = cv2.threshold(im_j, 127, 255, cv2.THRESH_BINARY)
kernel = np.ones((7,7), np.uint8)

closing_j = cv2.morphologyEx(im_j, cv2.MORPH_CLOSE, kernel)
plot_comparison(im_j, closing_j, 'closing opencv')

## 2.5. Морфологический градиент
Морфологический градиент вычисляется как разница между дилатацией и эрозией изображения. 

In [ ]:
gradient = cv2.morphologyEx(closing_j, cv2.MORPH_GRADIENT, kernel)

plot_comparison(closing_j, gradient, 'closing opencv')

## 2.6. Tophat-фильтрация
White_tophat преобразования изображения определяется как изображение за вычетом его морфологического размыкания. Эта операция возвращает яркие точки изображения, которые меньше структурного элемента.

In [ ]:
phantom = orig_phantom.copy()
phantom[340:350, 200:210] = 255
phantom[100:110, 200:210] = 0

w_tophat = white_tophat(phantom, selem)
plot_comparison(phantom, w_tophat, 'white tophat')

С помощью tophat преобразования можно выполнять фильтрацию. Например, можно выполнять удаление мелких объектов на изображениях в градациях серого. В данном случае воспользуемся преобразование white tophat, которое определяется как разница между входным изображением и его размыканием.

In [ ]:
image = color.rgb2gray(data.hubble_deep_field())[:500, :500]

selem_th =  disk(2)
res = white_tophat(image, selem_th)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20, 8))
ax[0].set_title('Original')
ax[0].imshow(image, cmap='gray')
ax[1].set_title('White tophat')
ax[1].imshow(res, cmap='gray')
ax[2].set_title('Complementary')
ax[2].imshow(image - res, cmap='gray')

plt.show()

Применение Black Tophat. Black_tophat изображения определяется как его морфологическое замыкание за вычетом исходного изображения. Эта операция возвращает темные пятна изображения, которые меньше структурного элемента.

In [ ]:
b_tophat = black_tophat(phantom, selem)
plot_comparison(phantom, b_tophat, 'black tophat')

## 2.7. Формирование остова
Формирование остова используется для уменьшения каждого связного компонента в двоичном изображении до скелета объекта шириной в один пиксель. Важно отметить, что эта операция выполняется только для двоичных изображений.  

Формирование скелета объекта может быть полезно для извлечения признаков и / или представления топологии объекта.

Скелетонизация работает на основе последовательных проходов изображения. На каждом проходе граничные пиксели идентифицируются и удаляются при условии, что они не нарушают связность соответствующего объекта.

In [ ]:
horse = data.horse()

sk = skeletonize(horse == 0)
plot_comparison(horse, sk, 'skeletonize')

__Подходы к построению остова__
Метод skeletonize работает, выполняя последовательные проходы изображения, удаляя пиксели на границах объекта. Это продолжается до тех пор, пока больше не удастся удалить пиксели. Изображение коррелируется с маской, которая присваивает каждому пикселю номер в диапазоне [0… 255], соответствующий каждому возможному шаблону из его 8 соседних пикселей. Затем используется таблица поиска для присвоения пикселям значения 0, 1, 2 или 3, которые выборочно удаляются во время итераций.

Существует метод построения остова по методу Lee (skeletonize (..., method = 'lee')), который использует структуру данных октодерева для исследования окрестности пикселя размером 3x3х3. Алгоритм выполняется путем итеративного прохождения по изображению и удаления пикселей на каждой итерации, пока изображение не перестанет изменяться. Каждая итерация состоит из двух шагов: во-первых, составляется список кандидатов на удаление; затем пиксели из этого списка последовательно перепроверяются, чтобы лучше сохранить связность изображения.

Обратите внимание, что метод Ли разработан для использования с трехмерными изображениями и выбирается для них автоматически. В иллюстративных целях мы применяем этот алгоритм к двухмерному изображению.

In [ ]:
blobs = data.binary_blobs(200, blob_size_fraction=.2,
                          volume_fraction=.35, rng=1)

skeleton = skeletonize(blobs)
skeleton_lee = skeletonize(blobs, method='lee')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(blobs, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].set_title('skeletonize')
ax[1].axis('off')

ax[2].imshow(skeleton_lee, cmap=plt.cm.gray)
ax[2].set_title('skeletonize (Lee 94)')
ax[2].axis('off')

fig.tight_layout()
plt.show()

Построение средней оси объекта - это получение совокупности всех точек, имеющих более одной ближайшей точки на границе объекта. Среднюю ось объекта часто называют топологическим каркасом, потому что это каркас объекта шириной 1 пиксель с той же связностью, что и исходный объект.

Здесь мы используем преобразование средней оси для вычисления ширины объектов переднего плана. Поскольку функция medial_axis возвращает преобразование расстояния в дополнение к средней оси (с аргументом return_distance = True), с помощью этой функции можно вычислить расстояние до фона для всех точек медиальной оси. Это дает оценку локальной ширины объектов.

Для объекта с меньшим количеством ветвей предпочтительнее использовать построение остова с помощью метода skeletonize.

In [ ]:
# Generate the data
blobs = data.binary_blobs(200, blob_size_fraction=.2,
                          volume_fraction=.35, rng=1)

# Compute the medial axis (skeleton) and the distance transform
skel, distance = medial_axis(blobs, return_distance=True)

# Compare with other skeletonization algorithms
skeleton = skeletonize(blobs)
skeleton_lee = skeletonize(blobs, method='lee')

# Distance to the background for pixels of the skeleton
dist_on_skel = distance * skel

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(blobs, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

ax[1].imshow(dist_on_skel, cmap='magma')
ax[1].contour(blobs, [0.5], colors='w')
ax[1].set_title('medial_axis')
ax[1].axis('off')

ax[2].imshow(skeleton, cmap=plt.cm.gray)
ax[2].set_title('skeletonize')
ax[2].axis('off')

ax[3].imshow(skeleton_lee, cmap=plt.cm.gray)
ax[3].set_title("skeletonize (Lee 94)")
ax[3].axis('off')

fig.tight_layout()
plt.show()

## 2.8. Морфологическое утончение
Морфологическое утончение работает по тому же принципу, что и скелетонизация: выполняется удаление пикселей с границ на каждой итерации, пока ни один из них не может быть удален без изменения связности. Различные правила удаления могут ускорить утончение и привести к получению разных окончательных остовов.

Функция морфологического утончения принимает необязательный аргумент max_num_iter, чтобы ограничить количество итераций прореживания и, таким образом, создать относительно более толстый каркас.

In [ ]:
image = invert(data.horse())
skeleton = skeletonize(image)
thinned = thin(image)
thinned_partial = thin(image, max_num_iter=25)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].set_title('skeleton')
ax[1].axis('off')

ax[2].imshow(thinned, cmap=plt.cm.gray)
ax[2].set_title('thinned')
ax[2].axis('off')

ax[3].imshow(thinned_partial, cmap=plt.cm.gray)
ax[3].set_title('partially thinned')
ax[3].axis('off')

fig.tight_layout()
plt.show()

## 2.9. Формирование выпуклой оболочки
Выпуклая оболочка изображения - это набор пикселей, включенных в наименьший выпуклый многоугольник, который окружает все белые пиксели входного изображения. Обратите внимание, что эта операция также выполняется с бинарными изображениями.

In [ ]:
horse = data.horse()
hull1 = convex_hull_image(horse == 0)
plot_comparison(horse, hull1, 'convex hull')

Как показано на рисунке, convx_hull_image дает самый маленький многоугольник, который полностью покрывает белый цвет или значение True на изображении.

Если мы добавим к изображению небольшую зернистость, мы увидим, как выпуклая оболочка адаптируется, чтобы охватить это зерно

In [ ]:
horse_mask = horse == 0
horse_mask[45:50, 75:80] = 1

hull2 = convex_hull_image(horse_mask)
plot_comparison(horse_mask, hull2, 'convex hull')

In [ ]:
hull_diff = img_as_float(hull2.copy())
hull_diff[horse_mask] = 2

fig, ax = plt.subplots()
ax.imshow(hull_diff, cmap=plt.cm.gray)
ax.set_title('Difference')
plt.show()

## 3. Сегментация

## 3.1. Пороговая сегментация
## 3.1.1. Базовая сегментация на основе бинаризации по глобальному яркостному порогу
__Метод Оцу__ вычисляет «оптимальный» порог (отмечен красной линией на гистограмме ниже) путем максимизации дисперсии между двумя классами пикселей, которые разделены пороговым значением. Равным образом этот порог минимизирует внутриклассовую дисперсию.

In [ ]:
def show_binary(image, binary):
    fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title('Original')
    ax[0].axis('off')

    ax[1].hist(image.ravel(), bins=256)
    ax[1].set_title('Histogram')
    ax[1].axvline(thresh, color='r')

    ax[2].imshow(binary, cmap=plt.cm.gray)
    ax[2].set_title('Thresholded')
    ax[2].axis('off')

    plt.show()

In [ ]:
image = data.coins()
thresh = threshold_otsu(image)
binary = image > thresh

show_binary(image, binary)

Негативное влияние неравномерной освещенности на результат сегментации по порогу.

In [ ]:
fig, ax = try_all_threshold(image, figsize=(10, 8), verbose=False)
plt.show()

## 3.1.2. Устранение неравномерности освещения
__Алгоритм катящегося мяча для оценки интенсивности фона__.
Алгоритм катящегося мяча оценивает интенсивность фона изображения в оттенках серого в случае неравномерной экспозиции. Он часто используется в биомедицинской обработке изображений и был впервые предложен Стэнли Р. Стернбергом в 1983 году. 

Алгоритм работает как фильтр и интуитивно понятен. Мы представляем изображение как поверхность, на которой блоки единичного размера наложены друг на друга вместо каждого пикселя. Количество блоков и, следовательно, высота поверхности определяется интенсивностью пикселя. Чтобы получить интенсивность фона в желаемом (пиксельном) положении, мы представляем погружение шара под поверхность в желаемом месте. Когда мяч полностью покрыт блоками, вершина шара определяет интенсивность фона в этой позиции. Затем мы можем катать этот шар под поверхностью, чтобы получить значения фона для всего изображения.

Scikit-image реализует обобщенную версию этого алгоритма, который позволяет вам использовать произвольные формы в качестве ядра и работает с n-мерными изображениями. Это позволяет напрямую фильтровать изображения RGB или фильтровать стеки изображений по любому (или всем) пространственным измерениям.

In [ ]:
background = restoration.rolling_ball(image)
image_result = image - background
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')
ax[0].axis('off')

ax[1].imshow(background, cmap='gray')
ax[1].set_title('Background')
ax[1].axis('off')

ax[2].imshow(image_result, cmap='gray')
ax[2].set_title('Result')
ax[2].axis('off')

fig.tight_layout()
plt.show()

In [ ]:
thresh = threshold_otsu(image_result)
binary = image_result > thresh

show_binary(image_result, binary)

## 3.1.3. Локальные методы бинаризации
Для учета эффектов неравномерности освещенности предлагается применять специализированные методы бинаризации. __Пороги Ниблака и Саувола__ - это локальные методы определения пороговых значений, которые полезны для изображений с неоднородным фоном, особенно при распознавании текста. Вместо расчета единого глобального порога для всего изображения, несколько пороговых значений вычисляются для каждого пикселя с использованием определенных формул, которые учитывают среднее значение и стандартное отклонение локальной окрестности (определяемой окном с центром вокруг пикселя).

In [ ]:
image = data.page()
binary_global = image > threshold_otsu(image)

window_size = 25
thresh_niblack = threshold_niblack(image, window_size=window_size, k=0.8)
thresh_sauvola = threshold_sauvola(image, window_size=window_size)

binary_niblack = image > thresh_niblack
binary_sauvola = image > thresh_sauvola

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.imshow(image, cmap=plt.cm.gray)
plt.title('Original')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title('Global Threshold')
plt.imshow(binary_global, cmap=plt.cm.gray)
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(binary_niblack, cmap=plt.cm.gray)
plt.title('Niblack Threshold')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(binary_sauvola, cmap=plt.cm.gray)
plt.title('Sauvola Threshold')
plt.axis('off')

plt.show()

## 3.1.4. Обработка с несколькими порогами
Множественные пороги Оцу - это алгоритм пороговой обработки, который используется для разделения пикселей входного изображения на несколько различных классов, каждый из которых получается в соответствии с интенсивностью уровней серого в изображении.

Multi-Otsu рассчитывает несколько пороговых значений, определяемых количеством желаемых классов. По умолчанию количество классов равно 3: для получения трех классов алгоритм возвращает два пороговых значения. Они представлены красными линиями на гистограмме ниже. 

In [ ]:
image = data.camera()

# Applying multi-Otsu threshold for the default value, generating
# three classes.
thresholds = threshold_multiotsu(image)

# Using the threshold values, we generate the three regions.
regions = np.digitize(image, bins=thresholds)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plotting the original image.
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original')
ax[0].axis('off')

# Plotting the histogram and the two thresholds obtained from
# multi-Otsu.
ax[1].hist(image.ravel(), bins=255)
ax[1].set_title('Histogram')
for thresh in thresholds:
    ax[1].axvline(thresh, color='r')

# Plotting the Multi Otsu result.
ax[2].imshow(regions, cmap='jet')
ax[2].set_title('Multi-Otsu result')
ax[2].axis('off')

plt.subplots_adjust()

plt.show()

In [ ]:
man_mask = np.int8(regions == 0)
plt.imshow(man_mask, cmap="gray")
plt.show()

In [ ]:
selem = disk(6)
opened = opening(man_mask, selem)

plt.imshow(opened, cmap="gray")
plt.show()

In [ ]:
opened_mask = opened == 1
hull2 = convex_hull_image(opened_mask)
hull_diff = img_as_float(hull2.copy())
hull_diff[opened] = 1
resulting = hull_diff * image

fig, ax = plt.subplots()
ax.imshow(resulting, cmap=plt.cm.gray)
plt.show()

## 3.2. Детектор границ Canny
Фильтр Кэнни - это многоступенчатый алгоритм детектирования границ. Данный фильтр основан на производной Гаусса для оценки интенсивности градиентов. Гауссовый фильтр уменьшает эффект шума, присутствующего в изображении. Затем потенциальные границы прореживаются до примитивов толщиной в 1 пиксель путем удаления немаксимумов градиента. Наконец, краевые пиксели сохраняются или удаляются с использованием порогового значения гистерезиса.

Canny имеет три настраиваемых параметра: ширину гауссиана (чем шумнее изображение, тем больше ширина), нижний и верхний пороги для порогового значения гистерезиса.

1. Поскольку обнаружение краев чувствительно к шуму в изображении, первым этапом является удаление шума в изображении с помощью гауссовского фильтра 5x5.  
  
2. Нахождение градиента интенсивности изображения. Сглаженное изображение фильтруется с помощью ядра Собеля как в горизонтальном, так и в вертикальном направлении, чтобы получить первую производную в горизонтальном направлении (Gx) и вертикальном направлении (Gy). Из этих двух изображений мы можем найти градиент и направление границ для каждого пикселя:
$$ Edge\_Gradient \; (G) = \sqrt{G_x^2 + G_y^2} $$

$$ Angle \; (\theta) = \tan^{-1} \bigg(\frac{G_y}{G_x}\bigg)$$
3. Подавление немаксимумов. После получения величины и направления градиента выполняется полное сканирование изображения для удаления любых нежелательных пикселей, которые могут не принадлежать границам. Для этого проверяется пиксель: является ли он локальным максимумом в окрестности в направлении градиента.
<img src="https://docs.opencv.org/master/nms.jpg" width="300" height="300">  
Точка А находится на краю (в вертикальном направлении). Направление градиента перпендикулярно краю. Точки B и C находятся в направлениях градиента. Таким образом, точка A сравнивается с точками B и C, чтобы оценить, образует ли она локальный максимум. В противном случае, значение подавляется (обнуляется). В результате, мы получаем двоичное изображение с «тонкими границами».
  
4. Оценка порогового значения гистерезиса. На этом этапе решается, какие из границ действительно являются границами, а какие нет. Для этого нам нужны два пороговых значения: minVal и maxVal. Любые края с градиентом интенсивности больше maxVal обязательно будут краями, а те, что ниже minVal, обязательно не будут краями. Те границы, что находятся между этими двумя порогами, классифицируются как границы в зависимости от их связности. Если они соединены с пикселями с четкими границами, они считаются частью границ. В противном случае они также отбрасываются.
<img src="https://docs.opencv.org/master/hysteresis.jpg" width="400" height="400"> 
В указанном примере граница A выше maxVal, поэтому считается "надежной границей". Хотя край C ниже maxVal, он соединен с ребром A, поэтому он также считается допустимым, и мы получаем полную кривую. Но ребро B, хотя оно выше minVal и находится в той же области, что и ребро C, не связано с каким-либо "точным краем", поэтому оно отбрасывается. На этом этапе также удаляются мелкие шумы.  

__Пример использования детектора границ Canny в библиотеке OpenCV__

In [ ]:
img = cv2.imread('messi.jpg', 0)
edges = cv2.Canny(img, 100, 200)

fig, ax = plt.subplots(ncols=2, figsize=(20, 8))
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

__Пример использования детектора границ Canny в библиотеке Scikit-image__  
Зададим размытое изображение с шумом

In [ ]:
im = np.zeros((128, 128))
im[32:-32, 32:-32] = 1

im = ndi.rotate(im, 15, mode='constant')
im = ndi.gaussian_filter(im, 4)
im += 0.2 * np.random.random(im.shape)

Применим детектор границ Canny с различными значениями sigma промежуточного Гауссова фильтра

In [ ]:
edges1 = feature.canny(im)
edges2 = feature.canny(im, sigma=3)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(8, 3),
                                    sharex=True, sharey=True)

ax1.imshow(im, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title('noisy image', fontsize=20)

ax2.imshow(edges1, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title(r'Canny filter, $\sigma=1$', fontsize=20)

ax3.imshow(edges2, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title(r'Canny filter, $\sigma=3$', fontsize=20)

fig.tight_layout()

plt.show()

## 3.3. Сегментирование на основе контуров и границ
## 3.3.1. Поиск прямых линий на основе преобразования Хафа
Преобразование Хафа в его простейшей форме - это метод обнаружения прямых.

В следующем примере мы создаем изображение с пересечением линии. Затем мы используем преобразование Хафа, чтобы исследовать пространство параметров для прямых линий, которые могут проходить через изображение.

Обычно линии параметризуются как y = mx + c с градиентом m и точкой пересечения y c. Однако это означало бы, что m стремится к бесконечности для вертикальных линий. Вместо этого мы строим отрезок, перпендикулярный линии, ведущий в начало координат. Линия представлена длиной этого сегмента r и углом, который он составляет с осью x, θ.

Преобразование Хафа создает массив гистограмм, представляющий пространство параметров (то есть матрицу M × N, для M различных значений радиуса и N различных значений θ). Для каждой комбинации параметров, r и θ, мы затем находим количество ненулевых пикселей во входном изображении, которые будут располагаться близко к соответствующей строке, и соответствующим образом увеличиваем массив в позиции (r, θ).

Локальные максимумы на полученной гистограмме указывают параметры наиболее вероятных линий. В нашем примере максимумы наблюдаются под углом 45 и 135 градусов, что соответствует углам вектора нормали каждой линии.

Другой подход - это прогрессивное вероятностное преобразование Хафа. Такой подход основан на предположении, что использование случайного подмножества точек голосования дает хорошее приближение к фактическому результату, и что линии могут быть извлечены во время процесса голосования путем обхода связанных компонентов. Это возвращает начало и конец каждого сегмента линии.

Функция probabilistic_hough имеет три параметра: общий порог, который применяется к аккумулятору Хафа, минимальная длина строки и промежуток между строками, который влияет на объединение строк. В приведенном ниже примере мы находим строки длиной более 10 пикселей с зазором менее 3 пикселей.

In [ ]:
# Constructing test image
image = np.zeros((200, 200))
idx = np.arange(25, 175)
image[idx, idx] = 255
image[line(45, 25, 25, 175)] = 255
image[line(25, 135, 175, 155)] = 255

# Classic straight-line Hough transform
# Set a precision of 0.5 degree.
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(image, theta=tested_angles)

In [ ]:
# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

angle_step = 0.5 * np.diff(theta).mean()
d_step = 0.5 * np.diff(d).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          d[-1] + d_step, d[0] - d_step]
ax[1].imshow(np.log(1 + h), extent=bounds, cmap=cm.gray, aspect=1 / 1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap=cm.gray)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

plt.tight_layout()
plt.show()

## 3.3.2. Круговое преобразование Хафа
Преобразование Хафа в его простейшей форме - это метод обнаружения прямых линий, но его также можно использовать для обнаружения кругов или эллипсов. Алгоритм предполагает, что границы на изображении уже обнаружены, а шумы или отсутствующие точки/разрыва устранены.

### Алгоритм поиска круговых объектов
Допустим на белом фоне располагается черный круг. Для начала мы задаем диапазон радиусов, по которым могут быть построен потенциальный круг. Этот круг наносится на каждый черный пиксель исходного изображения, и координаты этого круга учитываются в аккумуляторе. Далее подбирается положение центра круга, которое получает наивысший балл в аккумуляторе.  
Обратите внимание, что размер аккумуляторного объекта больше, чем у исходного изображения, чтобы определять центры кругов за пределами кадра.

### Пример обнаружения кругов с помощью Scikit-image
В следующем примере преобразование Хафа используется для определения положения монет.

In [ ]:
# Load picture and detect edges
image = img_as_ubyte(data.coins()[160:230, 70:270])
edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)

fig, ax = plt.subplots(ncols=2, figsize=(20, 8))
plt.subplot(121),plt.imshow(image,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Detect two radii
hough_radii = np.arange(20, 35, 2)
hough_res = hough_circle(edges, hough_radii)

In [ ]:
plt.imshow(hough_res[1],cmap = 'gray')
plt.title('Hough Image')
plt.show()

In [ ]:
# Select the most prominent 3 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=3)

In [ ]:
# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(image)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius,
                                    shape=image.shape)
    image[circy, circx] = (220, 20, 20)

ax.imshow(image, cmap=plt.cm.gray)
plt.show()

Получены координаты центров и радиусов объектов на изображении

In [ ]:
cx, cy, radii

### Алгоритм поиска эллипсов
Алгоритм берет две разные точки, принадлежащие эллипсу. Предполагается, что это главная ось. Цикл по всем другим точкам определяет, насколько эллипс включает их. Хорошее совпадение соответствует высоким значениям аккумулятора. По пиковым значениям аккумулятора выбирается значение, определяющее локализацию эллипса.

In [ ]:
# Load picture, convert to grayscale and detect edges
image_rgb = data.coffee()[0:220, 160:420]
image_gray = color.rgb2gray(image_rgb)
edges = canny(image_gray, sigma=2.0,
              low_threshold=0.55, high_threshold=0.8)

fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
plt.subplot(121),plt.imshow(image_gray,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# Perform a Hough Transform
# The accuracy corresponds to the bin size of a major axis.
# The value is chosen in order to get a single high accumulator.
# The threshold eliminates low accumulators
result = hough_ellipse(edges, accuracy=20, threshold=250,
                       min_size=100, max_size=120)
result.sort(order='accumulator')

# Estimated parameters for the ellipse
best = list(result[-1])
yc, xc, a, b = [int(round(x)) for x in best[1:5]]
orientation = best[5]

In [ ]:
# Draw the ellipse on the original image
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
image_rgb[cy, cx] = (0, 0, 255)
# Draw the edge (white) and the resulting ellipse (red)
edges = color.gray2rgb(img_as_ubyte(edges))
edges[cy, cx] = (250, 0, 0)

fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(10, 4),
                                sharex=True, sharey=True)

ax1.set_title('Original picture')
ax1.imshow(image_rgb)

ax2.set_title('Edge (white) and result (red)')
ax2.imshow(edges)

plt.show()

### Применение библиотеки OpenCV

In [ ]:
img = img_as_ubyte(data.coins()[160:230, 70:270])
cimg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 30,
                            param1=65, param2=40, minRadius=10, maxRadius=35)
circles = np.uint16(np.around(circles))

for i in circles[0, :]:
    # draw the outer circle
    cv2.circle(cimg, (i[0],i[1]), i[2], (0,255,0), 2)
    # draw the center of the circle
    cv2.circle(cimg, (i[0],i[1]), 2, (0,0,255), 3)

In [ ]:
plt.imshow(cimg)
plt.show()

## 3.4. Сегментация на основе водораздела
Сегментация на основе водораздела - это классический алгоритм, используемый для сегментации, то есть для разделения различных объектов на изображении.

Любое изображение в оттенках серого можно рассматривать как топографическую поверхность, где высокая интенсивность обозначает пики и холмы, а низкая интенсивность обозначает впадины.

Начиная с определяемых пользователем маркеров, алгоритм водораздела обрабатывает значения пикселей как местную топографию (высоту). Алгоритм затопляет бассейны от маркеров до тех пор, пока бассейны, относящиеся к разным маркерам (уровням интенсивности), не встретятся на линиях водоразделов. Во многих случаях маркеры выбираются в качестве локальных минимумов изображения, из которых затопляются бассейны.

### Процесс сегментации, реализуемой в библиотеке Scikit-image. 
В приведенном ниже примере необходимо разделить два перекрывающихся круга. Для этого вычисляется изображение, представляющее собой расстояние до фона. Максимумы этого расстояния выбраны в качестве маркеров, и затопление бассейнов от таких маркеров разделяет два круга вдоль линии водораздела. Подобный подход позволяет проводить сегментацию перекрывающихся объектов.

Сгенерируем два перекрывающихся круга.

In [ ]:
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

plt.imshow(image, cmap=plt.cm.gray)
plt.title('Overlapping objects')
plt.show()

Определим центры объектов

In [ ]:
distance = ndi.distance_transform_edt(image)
coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)

mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True

plt.imshow(mask, cmap=plt.cm.gray)
plt.title('Mask with centers')
plt.show()

In [ ]:
plt.imshow(-distance, cmap=plt.cm.gray)
plt.show()

In [ ]:
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=image)

plt.imshow(labels, cmap=plt.cm.gray)
plt.title('Separated objects')
plt.show()

### Процесс сегментации, реализуемый в библиотеке OpenCV
Рассмотрим изображение монет, которые соприкасаются друг с другом.

In [ ]:
img = cv2.imread('coins.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

fig, ax = plt.subplots(ncols=3, figsize=(20, 8))
plt.subplot(131),plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(132),plt.imshow(gray, cmap='gray')
plt.title('Gray Image'), plt.xticks([]), plt.yticks([])

plt.subplot(133),plt.imshow(thresh, cmap='gray')
plt.title('Binary Image'), plt.xticks([]), plt.yticks([])
plt.show()

Теперь нам нужно удалить шумы на изображении. Для этого мы можем использовать морфологическое размыкание. Чтобы удалить небольшие отверстия в объекте, мы можем использовать морфологическое замыкание. После выполнения данных операций мы точно знаем, что область рядом с центром объекта является передним планом, а область далеко от объекта - фоном. Единственная область, в которой мы не уверены, - это граничная область монет.

Итак, нам нужно извлечь область, в которой мы уверены, что это монеты. Эрозия удаляет граничные пиксели. Это сработало бы, если бы предметы не касались друг друга. Но поскольку они касаются друг друга, еще одним хорошим вариантом было бы найти преобразование расстояния и применить правильный порог. Затем нужно найти область, в которой точно не располагаются монеты. Для этого применяем дилатацию. Дилатация увеличивает границу объекта до фона. Таким образом, мы можем убедиться, что любая область фона в результате действительно является фоном.

In [ ]:
# noise removal
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# sure background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(20, 8))
plt.subplot(131), plt.imshow(cv2.cvtColor(sure_fg, cv2.COLOR_BGR2RGB))
plt.title('Foreground - White'), plt.xticks([]), plt.yticks([])

plt.subplot(132), plt.imshow(sure_bg, cmap ='gray')
plt.title('Background - Black'), plt.xticks([]), plt.yticks([])

plt.subplot(133), plt.imshow(unknown, cmap ='gray')
plt.title('Unknown region - White'), plt.xticks([]), plt.yticks([])
plt.show()

Теперь мы точно знаем, в каком регионе находятся монеты, где располагается фон. 
Далее мы создаем маркер (это массив того же размера, что и исходное изображение, но с типом данных int32) и маркируем области внутри него. Области, которые мы знаем наверняка (будь то передний план или фон), помечены любыми положительными целыми числами (разными целыми числами), а область, которую мы точно не знаем, просто оставляется равной нулю. Для этого мы используем cv.connectedComponents(). Он помечает фон изображения цифрой 0, затем другие объекты помечаются целыми числами, начиная с 1.

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(14, 8))
plt.subplot(121), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')

plt.subplot(122), plt.imshow(markers, cmap="jet")
plt.title('Markers')

plt.show()

Применим алгоритм сегментации на основе водораздела

In [ ]:
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 0, 255]

fig, ax = plt.subplots(ncols=2, figsize=(14, 8))
plt.subplot(121), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')

plt.subplot(122), plt.imshow(markers, cmap="jet")
plt.title('Markers')

plt.show()

### Пример сегментации объектов с помощью skimage
Далее представлен пример сегментации изображений с построением меток объектов. Применяются следующие шаги:
- Установление пороговых значений бинаризации с помощью автоматического метода Оцу
- Заполнение маленьких разрывов с помощью морфологического замыкания
- Удаление артефактов соприкосновения границ объектов
- Измерение областей изображения для фильтрации мелких посторонних объектов

In [ ]:
image = data.coins()[50:-50, 50:-50]

# apply threshold
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))

fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
plt.subplot(121), plt.imshow(image, cmap='gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122), plt.imshow(bw, cmap='gray')
plt.title('Binary Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# remove artifacts connected to image border
cleared = clear_border(bw)

# label image regions
label_image = label(cleared)

# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0)
                                
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
plt.subplot(121), plt.imshow(cleared, cmap = 'gray')
plt.title('Cleared border Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122), plt.imshow(label_image, cmap = 'gray')
plt.title('Label Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for i, region in enumerate(regionprops(label_image)):
    
    # take regions with large enough areas
    if region.area >= 100:
        print(f"Object {i}: {region.bbox}")
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

## 3.5. Метод сегментации на основе извлечения признаков
## 3.5.1. Извлечение морфометрических признаков объектов

In [ ]:
image = np.zeros((600, 600))

rr, cc = ellipse(300, 350, 100, 220)
image[rr, cc] = 1

image = rotate(image, angle=15, order=0)

rr, cc = ellipse(100, 100, 60, 50)
image[rr, cc] = 1

label_img = label(image)
regions = regionprops(label_img)

plt.imshow(label_img)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image, cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    orientation = props.orientation
    x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
    y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
    x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
    y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length

    ax.plot((x0, x1), (y0, y1), '-r', linewidth=2.5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
    ax.plot(x0, y0, '.g', markersize=15)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-b', linewidth=2.5)

ax.axis((0, 600, 600, 0))
plt.show()

Мы используем skimage.measure.regionprops_table() для вычисления (выбранных) свойств для каждого региона

In [ ]:
props = regionprops_table(label_img, properties=('centroid',
                                                 'orientation',
                                                 'major_axis_length',
                                                 'minor_axis_length'))
pd.DataFrame(props)

## 3.5.2. Метод случайного леса для сегментации на основе локальных признаков random forests
Сегментация на основе обучения вычисляется с использованием локальных функций на основе локальной интенсивности, краев и текстур в разных масштабах. Предоставляемая пользователем маска используется для идентификации различных областей. Пиксели маски используются для обучения классификатора случайного леса из scikit-learn. Затем неразмеченные пиксели маркируются на основе прогноза классификатора.

In [ ]:
full_img = data.skin()

plt.imshow(full_img)
plt.show()

img = full_img[:900, :900]

# Build an array of labels for training the segmentation.
# Here we use rectangles but visualization libraries such as plotly
# can be used to draw a mask on the image.
training_labels = np.zeros(img.shape[:2], dtype=np.uint8)
training_labels[:130] = 1
training_labels[:170, :400] = 1
training_labels[600:900, 200:650] = 2
training_labels[330:430, 210:320] = 3
training_labels[260:340, 60:170] = 4
training_labels[150:200, 720:860] = 4

In [ ]:
sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                        intensity=True, edges=False, texture=True,
                        sigma_min=sigma_min, sigma_max=sigma_max,
                        channel_axis=-1)
features = features_func(img)

In [ ]:
features.shape

In [ ]:
plt.imshow(features[:,:,4], cmap="gray")

In [ ]:
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                             max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(training_labels, features, clf)
result = future.predict_segmenter(features, clf)

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 10))
ax[0].imshow(segmentation.mark_boundaries(img, result, mode='thick'))
ax[0].contour(training_labels)
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result)
ax[1].set_title('Segmentation')
fig.tight_layout()

## 3.5.3. Оценка важности признаков
Ниже мы рассмотрим важность различных функций, рассчитанную с помощью scikit-learn. Характеристики интенсивности имеют гораздо большее значение, чем особенности текстуры. Может возникнуть соблазн использовать эту информацию для уменьшения количества признаков, предоставляемых классификатору, чтобы сократить время вычислений. Однако это может привести к переобучению и ухудшению результатов на границе между областями.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
l = len(clf.feature_importances_)

feature_importance = (
        clf.feature_importances_[:l//3],
        clf.feature_importances_[l//3:2*l//3],
        clf.feature_importances_[2*l//3:])

sigmas = np.logspace(
        np.log2(sigma_min), np.log2(sigma_max),
        num=int(np.log2(sigma_max) - np.log2(sigma_min) + 1),
        base=2, endpoint=True)

for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[0].plot(sigmas, feature_importance[ch][::3], 'o', color=color)
    ax[0].set_title("Intensity features")
    ax[0].set_xlabel("$\\sigma$")

for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[1].plot(sigmas, feature_importance[ch][1::3], 'o', color=color)
    ax[1].plot(sigmas, feature_importance[ch][2::3], 's', color=color)
    ax[1].set_title("Texture features")
    ax[1].set_xlabel("$\\sigma$")

fig.tight_layout()

## 3.5.4. Применение обученной модели к новому изображению
Далее можно применить обученный классификатор к новым объектам

In [ ]:
img_new = full_img[:700, 900:]

plt.imshow(img_new)
plt.show()


features_new = features_func(img_new)
result_new = future.predict_segmenter(features_new, clf)


fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 10))
ax[0].imshow(segmentation.mark_boundaries(img_new, result_new, mode='thick'))
ax[0].set_title('Image')
ax[1].imshow(result_new)
ax[1].set_title('Segmentation')
fig.tight_layout()

plt.show()

## 3.6. K-Means Clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

image = cv2.imread('monarch.jpg')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.show()

In [ ]:
pixel_vals = image.reshape((-1,3))
pixel_vals = np.float32(pixel_vals)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)

k = 3
retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)
segmented_data = centers[labels.flatten()]

segmented_image = segmented_data.reshape((image.shape))

plt.imshow(segmented_image)
plt.show()

## 4. Практический пример отслеживания объекта
Воспользуемся:
- переходом в цветовую модель HSV
- созданием маски для объекта с заданным цветом
- морфологической обработкой полученной маски
- нахождением контура объекта
- вычислением наименьшей окружности, охватывающей объект
- сохранением буфера последних точек локализации объекта

In [ ]:
# Зададим параметры отслеживаемого объекта в пространстве HSV
colorLower = (24, 100, 100)
colorUpper = (44, 255, 255)
pts = deque(maxlen=64)
 
# Создадим объект камеры
camera = cv2.VideoCapture(0)
 
# Работа с камерой в цикле
while True:
    # Прочитаем текущий кадр
    (grabbed, frame) = camera.read()
  
    frame = imutils.resize(frame, width=600)
    frame = imutils.rotate(frame, angle=180)
    
    # Выполним переход в цветовую модель HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
 
    # Создадим маску для объекта с заданным цветом
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    
    # Морфологическая обработка маски
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Поиск контуров и центра объекта
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
 
    # Если был найден контур
    if len(cnts) > 0:
        # найти наибольший контур по маске, затем использовать его
        # для вычисления наименьшей окружности, охватывающей объект
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
 
        if radius > 10:
            # нарисовать окружность локализации объекта
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
 
    # обновить список точек
    pts.appendleft(center)
    
    # Цикл по всем текущем (64) точкам локализации
    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue
 
        # Нарисовать след объекта
        thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
 
    # Отобразить картинку на экране
    frame = imutils.rotate(frame, angle=180)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # выйти из цикла обработки изображения
    if key == ord("q"):
        break

# Освободить ресурс камеры и закрыть всплывающее окно
camera.release()
cv2.destroyAllWindows()

Пример классического метода сегментации:
https://stackoverflow.com/questions/57813137/how-to-use-watershed-segmentation-in-opencv-python